  <style>
    .container {
      display: flex;
      gap: 10px; /* Optional spacing between divs */
    }

    .box h1 {
      font-size: 50px;
      padding-top: 10px;
      font-weight: 600;
    }
  </style>
  <div class="container" style="">
    <div class="box">
      <h1>Planning for Robotics</h1>
    </div>
    <div class="box" align="right">
      <img src="images/template/course_logo.svg" alt="Planning for Robotics" height="70"/>
      &nbsp;&nbsp;&nbsp;&nbsp;
      <img src="images/template/logo_i6.svg" alt="Chair Logo" height="70"/>
      &nbsp;&nbsp;&nbsp;&nbsp;
    </div>
  </div>

# Exercise Sheet 4 - Task Planning

In [5]:
## Introduction
In this week’s lab, we change course away from what we previously grouped as "planning for motion", toward deliberative reasoning. I.e., reasoning what to do instead of reasoning how to do it. For this, we will explore classical planning using the STRIPS formalism and the PDDL language. Unlike motion or navigation planning, where plans operate over continuous or geometric spaces, classical planning works on symbolic representations of the world — sets of discrete facts that describe the current state, and action descriptions that model how the world can change. Regardless, we can still use some of the same techniques we've learned so far to solve our problems, namely we can still employ search. 

One of the standard modeling languages for this kind of planning is PDDL (Planning Domain Definition Language), which is based on a fragment of First-Order Logic (FOL). It describes actions with preconditions and effects and enables planning systems to generate valid sequences of actions (plans) that lead from an initial state to a goal state. The actions that we model in PDDL are usually abstracted to some level, omitting geometric parameters or other details relevant to the low-level execution. 

In this exercise, we will introduce key ideas of domain modeling, learn how to express problems in PDDL, and then build a planner to solve them. In the final part, we will explore planning as heuristic search, including an introduction to the delete-relaxation heuristic, one of the most widely used strategies in optimal and satisficing planners. We will make use of the Universal Planning Framework (UPF) to parse and work with PDDL, and optionally visualize planning domains. However, most of the reasoning will be implemented by you.

### Learning Objectives
- Understand the basics of STRIPS and its relation to First-Order Logic
- Learn how to define planning domains and problems in PDDL
- Implement a basic forward state-space planner
- Apply heuristic search and understand the role of relaxation-based heuristics


### Acknowledgments
Some of the contents are based on Hector's lecture on "Actions and Planning in AI".

SyntaxError: invalid character '’' (U+2019) (676452971.py, line 2)

In [6]:
import os
import sys
import math
import heapq
import itertools
import time
import random
from typing import List, Tuple, Dict, Optional, Set

import matplotlib.pyplot as plt
from graphviz import Digraph

from collections import deque, defaultdict, namedtuple

try:
    from unified_planning.shortcuts import *
    from unified_planning.io import PDDLReader
except ImportError as e:
    raise ImportError("Please install the 'unified-planning' package (e.g., via `pip install unified-planning[all]`)") from e


## Task 1: Introduction to AI Planning
Recall the types of representations we used in the previous exercise sheets. In motion planning, we have expressed our environment, our actions, and our goals in dense real-valued vector spaces. In navigation planning, we made use of discretisation and represented our space in some integer, 2-dimensional vector space. Both of these representations are suitable to answer the question of "how" to solve the respective problem, as planning in these spaces directly relates to actionable movements for the robot. However, if we want to solve more complex tasks, we often need to chain different "actions" and reason about the "what" first. 

Consider, for example, a robot tasked with helping you clean a kitchen. You might have some cups in your dishwasher that need to be put into a cupboard. Some dirty plates and glasses from your kitchen table need to be put into the dishwasher. In this example, many different items need to picked up, put down somewhere else, and moved across a room. Performing this kind of reasoning, e.g., in the configuration space, would lead to a drastic increase in the dimensionality of the space even in the trivial case, where order of operation is neglected. 

In most cases, however, order of operation can not be neglected. Some of the ordering might just reflect preferences or "common sense" like taking the clean objects out of the dishwasher first to avoid getting them dirty again. Others might reflect physical constraints like objects blocking access to other objects, doors that need to be opened, etc.

### From Logic to Language
In symbolic planning, we aim to describe and reason about the world using some logical abstraction. I.e., we use a declarative, logical representation of the world. Typically, the chosen representations are in some fragment of First-Order Logic (FOL). FOL provides a powerful framework to describe objects, their properties and the relations between them. You should be familiar with FOL from your course on mathematical logics, but here is a small recap.  

FOL, consists of a set of logical symbols, and non-logical symbols. The logical symbols include the logical connectors as well as quantifiers. The non-logical symbols include: $n$-ary predicates, describing the relations between elements, $n$-ary function symbols, and an infinite set of variables. Constants are $0$-ary functions. We call a construct of FOL a term, if its either a variable or an arbitrary stack of functions applied on variables. A well-formed formular is a construct that is built from the following rules:
- predicate symbols with terms as arguments are formulas.
- Equality between terms is a formula.
- Negation applied on a formula is a formula
- Binary connectives (e.g., and, or, implication, etc) applied on two formulas are a formula
- Quantifiers applied on one variable and one formula is a formula. 

Consider the following example of a WFF:
$$
\forall x \text{Cup}(x) \implies \exists y (\text{Dishwasher}(y) \wedge \text{In}(x,y))
$$
While predicates are intrinsically meaningless, the above expression might be used to express that "every cup is in some dishwasher", specifying a partial goal of the setting we introduced above. While we can express these things nicely with the tools of FOL, we often use certain subsets that we call languages. 

A first order language, is a set of symbols and syntactic rules that allow us to construct well-formed formulas in that language. Often, these languages introduce some restrictions on the expressiveness of the resulting formulas. I.e., they might remove quantifiers, or restrict the arity of predicates. These restrictions are often beneficial in terms of computability and completeness. Generally, a less expressive language is easier to deal with algorithmitcally. 

The STRIPS language (and its successor PDDL) are examples of such restricted FOL-languages. In their base version, they include the following simplifcations:
- No function symbols
- No explicit quantifiers (quantification is hwoever possible through parameter instantiation)
- Only conjunctions of positive literals in states (closed-world assumption)
- Actions are pre-defined and limited. 

STRIPS (Stanford Research Institute Problem Solver), was originally developed in the early 1970s as part of the Shakey robot project. Shakey was the first AI-driven robot that could autonomously perceive, navigation, and plan its actions. PDDL was introduced as a standardized modeling language for benchmarking of automated planners. It is largely based on STRIPS and makes use of the following concepts:
- **Predicates**: similar to standard FOL, e.g., at(robot, room1), holding(cup1), etc.
- **Objects**: named constant symbols, e.g., robot, room1, cup1, these are usually typed (which can be expressed in FOL as $1$-ary predicates) 
- **Operators (Actions)**: defined using parameters, preconditions, and effects, change the *state* of the world. 

In PDDL, a problem consists of a general domain file (describing the predicates and action schemas, and types), and a problem file (describing the specifics of the current instance like which objects are available, what is the initial state, and what is the desired goal state). 

One of the most famous examples in classical planning is the Blocksworld. Here, a set of blocks, arbitrarily placed on a table, must be stacked in the desired configuration by a robot arm. The domain for blocksworld can be described the following way using PDDL:

We have two types: block and table, that indicate what kind of objects we might have. the actions pick_up and put_down are for picking and putting blocks from and on tables, wheres stack and unstack allow us to pick and put blocks from and on other blocks. From a symbolic level, this abstraction is sufficient to express any problem of stacking blocks into a desired configuration. A specific example with 3 blocks might be written in PDDL as follows:

Here, our initial state is that of block A being stacked on block C, and block B by itself on the table. In our desired configuration we want to have a full stack of C on B on A. 



### From Language to States
What we have written down here using PDDL, translates to a formal model of classical planning where a problem $P$ is defined as:
$$
P = \langle F, I, O, G\rangle,
$$
where:
- $F$ is a set of propositional (ground) fluents (i.e., predicates like clear(A))
- $I \subseteq F$ is the set of ground fluents that are true in the initial state
- $G \subseteq F$ is the set of ground fluents that must be true in any goal state
- $O$ is a finite set of operators, where $o = \langle \text{pre}(o), \text{add}(o), \text{del}(o) \rangle \in O$, with $\text{pre}(o), \text{add}(o), \text{del}(o) \subseteq F$.

Each action $o$, transforms the state $s \subseteq F$ into a new state $s'$ by:
- Checking if $\text{pre}(o) \subseteq s$
- Removing $\text{del}(o)$ from $s$
- Adding $\text{add}(o)$ to $s$

You may notice that while we use variables in STRIPS/PDDL, the above formalism talks about ground fluents. This is an important difference in notation, can in practice however be ignored by "grounding out" the PDDL problem. 

We can now define a state model for a planning problem instance as
$S = \langle \mathcal{S}, s_0, \mathcal{S}_G, \text{Act}, A, f, c \rangle$, where:
- $\mathcal{S}$: corresponding to $F$ in our previous model
- $s_0$: initial state.
- $\mathcal{S}_G$: set of goal states, i.e., all states $s \subseteq S$ such that $G \subseteq s$.
- $\text{Act}$: set of actions, corresponding to $O$.
- $A(s)$: set of applicable actions in state $s$, i.e., all ground actions $o$ such that $\text{pre}(o) \subseteq s$.
- $f(a, s)$: deterministic transition function (applying add and del for $a$).
- $c(a, s)$: cost (often just 1).



### From States to Search
This state model formulation matches the structure of a directed graph. Finding a plan for the problem $S(P)$ in state model formulation, reduces to finding a path in a graph where the
- Nodes represent the states $s$ in the model
- Edges $(s, s')$ capture corresponding transitions $s' = f(a,s), a \in A(s)$.

These state models and graphs are implicitely given by $P$ and are exponential in the number of atoms in $F$. From previous exercises we know that once we have a graph, we can use general graph search algorithms to find a solution to our problem. In this specific example, the problem can be phrased as finding a path from the root $s_0$ to a goal $s \in \mathcal{S}_G$ by applying actions. However, due to the size, we will see that its critical to use some heuristic function to guide the search.

### Coding Task 1.1 - FOL Expressions
Translate this action into full First-Order Logic using quantifiers:

“If a robot is in a room and the room is connected to another, it can move there.”

move(r, room1, room2) ; \\\
r $\in$ Robot ; \\\
room1, room2 $\in$ Room ; \\\
(r in room1 $\land$ room1 connected room2) $\Rightarrow$ (r in room1 $\xor$ r in room2) ; \\\

### Coding Task 1.2 - Planning Problem Modelling
Translate the previous expression into a planning model with ground fluents and operators according to the $P = \langle F, I, O, G\rangle$ model. Provide some initial state and goal state by describing them as set of fluents. Define the add, del, and pre lists for the operators. Re-use the same robot room navigation problem from before as your base. 

In [2]:
# F = { ... }

# I = { ... }

# G = { ... }

# O = [
#   {
#     'name': 'move(r1, room1, room2)',
#     'pre': { ... },
#     'add': { ... },
#     'del': { ... }
#   },
#   ...
# ]

### Coding Task 1.3 - Model the Problem in PDDL
Translate the previous example into a fully fledged PDDL model. Recall that PDDL separates domain and problem knowledge. Thus you'll need to provide both parts separate from each other. For your problem, create one example for a robot that wants to move from one room to another room with at least two paths possible solution paths. 

### Coding Task 1.4 - Visualise the State Space
Visualize the state space for the above problem + domain description as a graph.

In [ ]:
# Das was Kevin in Gimp gemahlt hat danke kevin

## Task 2: PDDL Planning
We've now covered the basics of what makes a planning model for PDDL/STRIPS. We've also seen how we can generate a search graph for our planning problem and thus formulate the problem of generating a plan as one of finding a path between two nodes in the search graph. 

A planner is -- in a sense -- nothing more than a search algorithm over symbolic states.
In case of forward search, the planner tarts at $s_0 = I$ and iteratively expands child states to find a state $s$ where $G \subseteq s$. This search can be done using the standard graph search algorithms you are already familiar with from previous exercises or other classes:
- Breadth-first search (BFS)
- Uniform-cost search
- Depth-first search
- A* search (when using heuristics)

Two important practical considerations for implementing a modern planner are what kind of heuristic to choose, and how to represent the state space in a memory efficient way. We will look into heuristics in the next section and omit the topic of efficient representations for this course. Instead, we will work with the Universal Planning Framework (UPF) in this exercise. 

The UPF is a modern Python-based library that provides a unified interface for planning languages and solvers. It supports:
- Parsing PDDL domains and problems and exporting them to target planners.
- Interfacing with many common planners (e.g., Tarski, pyperplan, FastDownward).
- Tools to inspecting fluents, actions, and initial/goal states.

For a more detailed introduction refer to the following resources: 
- Website: https://github.com/aiplan4eu/unified-planning
- Documentation: https://unified-planning.readthedocs.io/

Parts of the exercise will only be solvable if you study the documentation, and the code in the repository on GitHub. This is intentional, as its representative of common situations when you're working with existing libraries or frameworks. 


### Coding Task 2.1
With the UPF, you can directly parse the PDDL domain and problem files from the previous tasks and inspect the parsing results. Example code for this is provided below. Explore some of the features of the UPF and familiarise yourself with the results from parsing. Think about how you could build a search tree from these representations. Take a special look at the representations of pre-conditions of actions.

In [91]:
from unified_planning.io import PDDLReader

reader = PDDLReader()
problem = reader.parse_problem("domain.pddl", "problem.pddl")

print("Fluents:", problem.fluents)
print("Actions:", problem.actions)
print("Initial state:", problem.explicit_initial_values.items())
print("Goals:", problem.goals)

Fluents: [bool at[r=robot, room=room], bool connected[from=room, to=room]]
Actions: [action move(robot r, room from, room to) {
    preconditions = [
      (at(r, from) and connected(from, to))
    ]
    effects = [
      at(r, from) := false
      at(r, to) := true
    ]
  }]
Initial state: dict_items([(at(r1, room1), true), (connected(room1, room2), true), (connected(room2, room3), true), (connected(room1, room3), true)])
Goals: [at(r1, room3)]


In [92]:
# define some helpful functions for later

def is_action_applicable(action_instance, state: Set[str]) -> bool:
    return True

def apply_action(action_instance, state: Set[str]) -> Set[str]:
    return state

def ground_actions(problem):
    return grounded_actions

from unified_planning.io import PDDLReader

reader = PDDLReader()
problem = reader.parse_problem("domain.pddl", "problem.pddl")

actions = ground_actions(problem)


condition (at(r, from) and connected(from, to))
condition (at(r, from) and connected(from, to))
condition (at(r, from) and connected(from, to))
condition (at(r, from) and connected(from, to))
condition (at(r, from) and connected(from, to))
condition (at(r, from) and connected(from, to))
condition (at(r, from) and connected(from, to))
condition (at(r, from) and connected(from, to))
condition (at(r, from) and connected(from, to))



### Coding Task 2.2
Now that you can represent states and apply actions, you’ll implement a simple forward-search planner that:
- Starts at the initial state.
- Explores successors using BFS by expanding all successors through the applicable actions.
- Returns the sequence of actions that leads to the goal state.

In [88]:
def strips_planner(problem) -> Optional[List[str]]:
    
    return None  # No plan found

In [89]:
from unified_planning.io import PDDLReader

reader = PDDLReader()
problem = reader.parse_problem("domain.pddl", "problem.pddl")

plan = strips_planner(problem)

if plan:
    print("Plan found:")
    for step in plan:
        print("  ", step)
else:
    print("No plan found.")

{'connected(room1, room2)', 'at(r1, room1)', 'connected(room2, room3)', 'connected(room1, room3)'}
frozenset({'connected(room1, room2)', 'at(r1, room1)', 'connected(room2, room3)', 'connected(room1, room3)'})
{frozenset({'connected(room1, room2)', 'at(r1, room1)', 'connected(room2, room3)', 'connected(room1, room3)'})}
move(r1, room1, room1)
condition (at(r, from) and connected(from, to))
move(r1, room1, room2)
condition (at(r, from) and connected(from, to))
move(r1, room1, room3)
condition (at(r, from) and connected(from, to))
move(r1, room2, room1)
condition (at(r, from) and connected(from, to))
move(r1, room2, room2)
condition (at(r, from) and connected(from, to))
move(r1, room2, room3)
condition (at(r, from) and connected(from, to))
move(r1, room3, room1)
condition (at(r, from) and connected(from, to))
move(r1, room3, room2)
condition (at(r, from) and connected(from, to))
move(r1, room3, room3)
condition (at(r, from) and connected(from, to))
❌ No plan found.


### Planning as Heuristic Search
We will now finally delve into heuristic-based forward search planning. In particular, we want to work with the $h_\text{max}$ heuristic, which indicates the number of steps required to get all atoms in $G$ to be true. To compute a heuristic for planning problems, often some form of simplified problem is used. Think back to the geometric planning problems: the Manhattan distance is an admissible heuristic obtained from a simplifcation of the space. We can find similar simplifications also for STRIPS/PDDL Planning problems. 

#### Delete Relaxation
The delete-relaxation (DR) was one of the first, widely established problem relaxations in classical planning and was the basis for several heuristics, notably $h^+$, $h_\text{max}$, and $h_\text{add}$.

The DR is defined as follows: for a STRIPS action $a$, we denote $a^+$ the corresponding *delete relaxed action*, for which $\text{pre}_{a^+}:=\text{pre}_{a}$, $\text{add}_{a^+}:=\text{add}_{a}$, $\text{del}_{a^+}:=\emptyset$. If we apply this relaxation onto all actions $a\in A$ of a STRIPS problem, we get the its corresponding *relaxed* planning task. 

There are two major insights to DR planning tasks:
- **delete relaxations are over-approximating**, i.e., any suitable plan for a non-relaxed planning problem, is a plan for the corresponding relaxed planning problem, with its actions replaced by the relaxed actions. 
- **delete relaxations are fully decomposable**, i.e., any plan $\pi_1$ that achives $G_1$ followed by any plan $\pi_2$ that achieves $G_2$, the plan $\pi_1, \pi_2$ achieves both $G_1$ and $G_2$. This is because the delete relaxation can only ever make more facts true, it can never render any task unsolvable. 

Unfortunately, finding an optimal plan for the relaxed problem, is about as difficult as solving the original planning problem optimally (both are NP-hard). Thus, we haven't found a shortcut for the actual problem. However, finding any plan for the relaxed problem is easy. This is due to its decomposability: To yield a plan for a goal $G$, we just need a plan $\pi_p$ for each atom $p \in G$. Once we have our plans for each $p$, we can just stack them to get our (non-optimal) plan for $G$. 

#### The $h_\text{max}$ Heuristic
The $h_\text{max}$ heuristic is one such heuristic derived from the DR of a planning problem. It is an admissible heuristic, thus optimal solutions can be derived through the solver. Generally it is considered an optimistic heuristic, i.e., it overestimates the quality of a given state and thus leads to non-optimal expansions in the search. 

We define the $h_\text{max}$ heuristic recursively for a state $s$, a problem $P$, and a goal $G$ as follows:
$$
h_\text{max}(s) := h_\text{max}(s, G) := 
\left\{
\begin{array}{ll}
0 & \text{if } g \subseteq s \\\\
\min\limits_{a \in A,\ g' \in \text{add}(a)} c(a) + h_{\text{max}}(s, \text{pre}(a)) & \text{if } g = \{g'\} \\\\
{\max\limits_{g' \in g}}\, h_{\text{max}}(s, \{g'\}) & \text{if } |g| > 1
\end{array}
\right.
$$

Since we assume uniform costs $c(a)=1$, intuitively speaking, the $h_\text{max}$ heuristic counts the number of steps to make all atoms in $G$ true in a current state. The lower the number, the *better* it is to expand the state, as it would mean less follow up expansions.  

However, especially by modern standards, it is not considered to be a very informative heuristic. The reason for this is that the underlying simplification takes little consideration for interdepencies in the problem, thus leading to many non-optimal expansions. 



### Coding Task 3.1
You will now implement your own estimator function for the $h_\text{max}$ heuristic and use A* or a similar heuristic search algorithm to solve the same planning problem as above. 

In [90]:
def h_max(state):
    return 0 # Replace with actual heuristic calculation

def heuristic_planner(problem) -> Optional[List[str]]:
    
    return None  # No plan found

In [ ]:
from unified_planning.io import PDDLReader

reader = PDDLReader()
problem = reader.parse_problem("domain.pddl", "problem.pddl")

plan = heuristic_planner(problem)

if plan:
    print("Plan found:")
    for step in plan:
        print("  ", step)
else:
    print("No plan found.")

### Coding Task 3.2 
Compare the planning times, number of expanded states, and heuristic computation times for both of your planner implementations.

## Submission Checklist
- [ ] All of the coding tasks are completed
- [ ] You can show the visualisations and animations as requested
- [ ] You can confidently provide answers to the questions posed in the tasks